<a href="https://colab.research.google.com/github/megha70265/Poetic-Text-Generation/blob/main/Poetic_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import tensorflow as tf

In [ ]:
filepath = tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(filepath, 'rb').read().decode(encoding='utf-8')

1115394/1115394 [==============================] - 0s 0us/step


In [ ]:
text=text.lower()
text = text[300000:800000]

In [ ]:
characters = sorted(set(text))

char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

A sorted set of characters is created. In python set no value appears more than once. We create two dictionaries one for accessing index of a character by charcter and second character by index. This way a representation for each charcter can be created.

In [ ]:
SEQ_LENGTH = 40
STEP_SIZE = 3

sentences = []
next_char = []

We define how long a sequence shall be and also how many characters we will step further to start the next sentence. What we try to do here is to take sentences and then save the next letter as the training data.

In [ ]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_char.append(text[i + SEQ_LENGTH])

In [ ]:
x = np.zeros((len(sentences), SEQ_LENGTH,
              len(characters)), dtype=bool)
y = np.zeros((len(sentences),
              len(characters)), dtype=bool)

for i, satz in enumerate(sentences):
    for t, char in enumerate(satz):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_char[i]]] = 1

## Building Recurent Network

In [ ]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

In [ ]:
model=Sequential()
model.add(LSTM(128,input_shape=(SEQ_LENGTH,len(characters))))

model.add(Dense(len(characters)))

model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.01))

model.fit(x, y, batch_size=256, epochs=4)

Epoch 1/4
1302/1302 [==============================] - 235s 179ms/step - loss: 2.5253
Epoch 2/4
1302/1302 [==============================] - 227s 174ms/step - loss: 2.1678
Epoch 3/4
1302/1302 [==============================] - 244s 188ms/step - loss: 2.0457
Epoch 4/4
1302/1302 [==============================] - 233s 179ms/step - loss: 1.9493


In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

It basically just picks one of the characters from the output. As parameters it takes the result of the prediction and a temperature. This temperature indicates how risky the pick shall be. If we have a high temperature, we will pick one of the less likely characters. A low temperature will cause a conservative choice.

In [ ]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

In [ ]:
print(generate_text(300,0.1))

xcuse that thou dost make in this delay
the warthe he mard the soull and the have the will wear here the ward the have the will the will sheer his the have the thee the sour the hand the will for the thee sears,
and the warther with the have the have the thee the will the ward the will the have the thee the will the will the will sheer th
